# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [46]:
import requests
import pandas as pd
import sqlite3

import time
#import tqdm

## Список объектов

In [33]:
%%time
offset_ = 1
#Все объекты не помещаются, поэтому лимит = 1000

limit_ = 1000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()

Wall time: 7.72 s


In [34]:
%%time
data = objects_data.get('data').get('list')
df = pd.json_normalize(data)
df.head()

Wall time: 80.8 ms


,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,objProblemFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
1,13399,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,0.00,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
3,13391,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,4293.67,1,0,...,Акционерное общество,АО,6.994210e+08,7701651356,NaN,NaN,NaN,NaN,NaN,NaN
4,23682,47,д Агалатово,10,10,54,2022-06-30,2748.15,1,0,...,Общество с ограниченной ответственностью,ООО,5.562608e+09,7806536079,"Жилой комплекс ""84 высота""",NaN,NaN,NaN,NaN,NaN


In [36]:
#Размер df

len(df)

1000

In [28]:
#Сохранение df в форматы xlsx, pickle

df.to_excel("df.xlsx")
df.to_pickle("df.pckl")

In [26]:
#Созздание коннекта к sqlite и создание таблицы test_table, в которую записываем наш df

con = sqlite3.connect('test_db.db')
df.to_sql(name='test_table', con=con)

In [27]:
#Проверка записи данных в таблицу test_table, селектим из неё и кладём в df2

df2 = pd.read_sql_query("""select * from test_table""",con=con)
df2.head()

,index,objId,rpdRegionCd,objAddr,objFloorMin,objFloorMax,objElemLivingCnt,objReady100PercDt,objSquareLiving,objCompensFundFlg,...,developer.orgForm.fullForm,developer.orgForm.shortForm,developer.companyGroup,developer.devInn,objCommercNm,complexShortName,developer.bankruptStage.bankruptStageCd,developer.bankruptStage.bankruptStageDesc,developer.bankruptStage.bankruptLawUrl,developer.orgBankruptMsgDttm
0,0,13397,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,6248.51,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
1,1,13399,72,"г Тюмень, ул Профсоюзная, д. 56",1,1,0,2024-12-31,0.00,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
2,2,13398,72,"г Тюмень, ул Профсоюзная, д. 56",7,10,43,2024-12-31,4812.98,0,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
3,3,13391,72,"г Тюмень, ул Профсоюзная",3,12,44,2022-06-30,4293.67,1,...,Акционерное общество,АО,6.994210e+08,7701651356,None,None,NaN,None,None,None
4,4,23682,47,д Агалатово,10,10,54,2022-06-30,2748.15,1,...,Общество с ограниченной ответственностью,ООО,5.562608e+09,7806536079,"Жилой комплекс ""84 высота""",None,NaN,None,None,None


In [29]:
#Закрытие коннекта

con.close()